Reference FaceAlignment Model
- https://paperswithcode.com/task/face-alignment
- https://github.com/zhenglinzhou/star (STAR Loss: Reducing Semantic Ambiguity in Facial Landmark Detection)
- https://github.com/huangyangyu/ADNet


| Dataset      | Best Model           | Paper | Code  | Compare |
|--------------|----------------------|-------|-------|---------|
| 300W         | STAR                 | 📄    | 🔗   |         |
| WFLW         | LDEQ                 | 📄    | 🔗   |         |
| COFW         | SH-FAN               | 📄    | 🔗   |         |
| AFLW-19      | FaRL-B (epoch 16)    | 📄    | 🔗   |         |
| AFLW2000-3D  | MNN+OR (reannotated) | 📄    | 🔗   |         |




# ***0. CONNECT GOOGLE DRIVE***

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ***1. WFLW DATASET AND STRUCTURE DATA FOR TRAINING***
https://wywu.github.io/projects/LAB/WFLW.html
https://datasets.activeloop.ai/docs/ml/datasets/wflw-dataset/


# ***1' Load Dataset form Kaggle***
- Sau khi processing, tôi push dataset zip file to Kaggle
https://www.kaggle.com/datasets/cngonngc/facealign-wflwds-adnetstarloss-proccolab-upkaggle

## Load Kaggle .json

In [2]:
!mkdir /content/drive/MyDrive/colab
%cd /content/drive/MyDrive/colab

# Upload kaggle.json file
from google.colab import files
files.upload()

#Check to make sure the kaggle.json file is present.
!ls -lha kaggle.json

# Install the Kaggle API client. (thư viện API của Kaggle)
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/ # The Kaggle API client expects this file to be in ~/.kaggle,

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json # Thay đổi quyền truy cập tệp kaggle.json để tránh cảnh báo khi khởi động công cụ Kaggle

mkdir: cannot create directory ‘/content/drive/MyDrive/colab’: File exists
/content/drive/MyDrive/colab


Saving kaggle.json to kaggle.json
-rw------- 1 root root 64 Jan  6 11:45 kaggle.json


## Download Dataset from Kaggle

In [3]:
# https://www.kaggle.com/datasets/cngonngc/facealign-wflwds-adnetstarloss-proccolab-upkaggle
# Link share: https://www.kaggle.com/datasets/cngonngc/facealign-wflwds-adnetstarloss-proccolab-upkaggle
%cd /content/drive/MyDrive/colab
!kaggle datasets download -d cngonngc/facealign-wflwds-adnetstarloss-proccolab-upkaggle    # -d is dataset, -c is competition

/content/drive/MyDrive/colab
100% 740M/742M [00:31<00:00, 29.3MB/s]
100% 742M/742M [00:31<00:00, 24.6MB/s]


In [4]:
# !unzip /content/drive/MyDrive/colab/facedetection-widerfacedataset-yolov5-zip -d /content/drive/MyDrive/colab # Unzip
!unzip /content/drive/MyDrive/colab/facealign-wflwds-adnetstarloss-proccolab-upkaggle.zip -d /content/drive/MyDrive/colab/ # Unzip, -d để điều hướng tới vị trí đặc biệt
# 5min-6min30s (GPU)

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/colab/dataset_for_STAR/image_dir/WFLW/WFLW_images/2--Demonstration/2_Demonstration_Political_Rally_2_206.jpg  
  inflating: /content/drive/MyDrive/colab/dataset_for_STAR/image_dir/WFLW/WFLW_images/2--Demonstration/2_Demonstration_Political_Rally_2_219.jpg  
  inflating: /content/drive/MyDrive/colab/dataset_for_STAR/image_dir/WFLW/WFLW_images/2--Demonstration/2_Demonstration_Political_Rally_2_22.jpg  
  inflating: /content/drive/MyDrive/colab/dataset_for_STAR/image_dir/WFLW/WFLW_images/2--Demonstration/2_Demonstration_Political_Rally_2_230.jpg  
  inflating: /content/drive/MyDrive/colab/dataset_for_STAR/image_dir/WFLW/WFLW_images/2--Demonstration/2_Demonstration_Political_Rally_2_243.jpg  
  inflating: /content/drive/MyDrive/colab/dataset_for_STAR/image_dir/WFLW/WFLW_images/2--Demonstration/2_Demonstration_Political_Rally_2_25.jpg  
  inflating: /content/drive/MyDrive/colab/dataset_for_STAR/image_dir/

In [5]:
!rm /content/drive/MyDrive/colab/facealign-wflwds-adnetstarloss-proccolab-upkaggle.zip  # xóa file .zip

# ***2. Model***

In [6]:
%cd /content/drive/MyDrive/colab
!git clone https://github.com/zhenglinzhou/star
# ModuleNotFoundError: No module named 'tensorboardX'
# Bị lỗi khi pip nên ko pip nữa  !pip install -r requirements.txt


/content/drive/MyDrive/colab
Cloning into 'star'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 72 (delta 8), reused 69 (delta 5), pack-reused 0
Receiving objects: 100% (72/72), 990.84 KiB | 12.54 MiB/s, done.
Resolving deltas: 100% (8/8), done.


## ***2.1 Wandb, Note Bug in training processing***

In [7]:
# Connect wandb
%cd /content/drive/MyDrive/colab/star
!pip install wandb
import wandb
wandb.login(key = "c8767797aae76cbcd389ff29929ace1ac3021161")    # key's DoanNgocCuong
wandb.init(project = "FaceAlignment_ADNetwithSTARloss", entity="doanngoccuong_nh")
# entity: người/nhóm thực hiện

/content/drive/MyDrive/colab/star
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin


## 2.3 Repair Fix Fork Git Clone and Use for Training
Sửa mã fork từ STAR, -> setup ở git cá nhân, load lại xài cho nhanh.
- Fix Bug requirements.txt
- Fix Bug`evalues, evectors = torch.linalg.eigh(covars, UPLO='L') # Doan Ngoc Cuong fix`
trong các file `# !grep -rnw './' -e 'symeig'  # fix bug `

Setup Wandb:
- Setup wandb.init() and wandb.finish() trong main.py
- Log training metrics trong lib/utility.py
- Log valid metrics và save best model trong trainer.py
- Thêm INFO lưu trữ save model trong lib/utility.py
- Add args.pretrained_weights vào main.py, sửa cách save_model with `net` thay vì `best_net` ở file `trainer.py`(best model save torch được log artifacts và lastest model được save torch ko log artifacts)

### 2.3.1 Set up my prepare repo

In [8]:
# Set up load my prepared Repo
# pip requirement.txt

%cd /content/drive/MyDrive/colab
!git clone https://github.com/DoanNgocCuongBKEGNH/FaceAlignment_WFLWds_ADNetwithSTARloss.git --branch WFLWds_ADNetwithSTARLoss
!pip install wandb
!pip install tensorboardX
# !pip install -r

/content/drive/MyDrive/colab
fatal: destination path 'FaceAlignment_WFLWds_ADNetwithSTARloss' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.0 MB/s eta 0:00:00


### 2.3.2 Train từ scratch

In [ ]:
# Xem file main.py to more inf --argument for cmd
# change max_epoch: ./conf/alignment.py:36:        self.max_epoch = 500
# batch_size để 4 (thay vì df: 128 - eval 32)
%cd /content/drive/MyDrive/colab/FaceAlignment_WFLWds_ADNetwithSTARloss
!python main.py --mode=train --device_ids=0 \
                --image_dir='/content/drive/MyDrive/colab/dataset_for_STAR/image_dir' \
                --annot_dir='/content/drive/MyDrive/colab/dataset_for_STAR/annot_dir' \
                --data_definition=WFLW \
                --batch_size=4 --val_batch_size=4

The output you've posted appears to be log entries from a model training and validation process. Here's a breakdown of the key parts:

1. `Val_net/Metric` and `Val_net_ema/Metric`: These are validation metrics for two different models or states of the model. "net" could be the current state of the model, and "net_ema" could be an Exponential Moving Average (EMA) version of the model, which is often used to stabilize and improve training.

2. `Metric 0`, `Metric 1`, ..., `Metric 11`: These represent different evaluation metrics or tests that are being logged. The numbers (0, 1, ..., 11) could correspond to different tasks or aspects of the model's predictions being evaluated.

3. `[NME nan, FR nan, AUC nan]`: These are specific metrics being reported. NME stands for Normalized Mean Error, FR for Failure Rate, and AUC for Area Under the Curve. `nan` indicates that, for some reason, the calculation for that metric resulted in an undefined number, which can happen if the predictions are incorrect in a way that makes the metric calculation impossible.

4. `Val/Best_Metric009 in this epoch: 0.134098`: This indicates the best metric obtained during validation for the specific metric index (9 in this case) for the current epoch. The value `0.134098` is the NME score, suggesting the error rate of predictions.

5. `model saved in this epoch`: Indicates that the model (or its state) has been saved during this epoch, likely because it achieved the best validation score thus far.

This information is useful for monitoring the training and validation process, identifying if and when the model might be failing to learn, and ensuring that the best model is retained for future use.

### 2.3.3 Train từ epoch 3-6
(bug: colab chỉ trên được 3epo - tầm 3h là đứng đột ngột như hết bộ nhớ - ko báo lỗi )

In [13]:
# Load weights epoch 3
run = wandb.init()
artifact = run.use_artifact('doanngoccuong_nh/FaceAlignment_ADNetwithSTARloss/ADNetSTARLoss_bestmodel:v6', type='model')
artifact_dir = artifact.download()

wandb: Downloading large artifact ADNetSTARLoss_bestmodel:v6, 236.07MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:14.1


In [ ]:
%cd /content/drive/MyDrive/colab/FaceAlignment_WFLWds_ADNetwithSTARloss
!python main.py --mode=train --device_ids=0 \
                --image_dir='/content/drive/MyDrive/colab/dataset_for_STAR/image_dir' \
                --annot_dir='/content/drive/MyDrive/colab/dataset_for_STAR/annot_dir' \
                --data_definition=WFLW \
                --batch_size=4 --val_batch_size=4 \
                --pretrained_weight='/content/drive/MyDrive/colab/artifacts/ADNetSTARLoss_bestmodel:v6/best_model_epoch3_valbest_metric0.0795.pkl'

'''
INFO
2024-01-06 05:53:00,937 INFO    : Loaded data
2024-01-06 05:53:00,938 INFO    : Optimizer type adam. Start training...
2024-01-06 05:53:00,938 INFO    : Train/Epoch: 0/6, Learning rate decays to [0.001], [Time Left: 00:00:00]eval_time: 0.00,
2024-01-06 05:53:00,939 INFO    : Train/Epoch: 1/6, Learning rate decays to [0.001], [Time Left: 00:00:00]eval_time: 0.00,
2024-01-06 05:53:00,940 INFO    : Train/Epoch: 2/6, Learning rate decays to [0.001], [Time Left: 00:00:00]eval_time: 0.00,
2024-01-06 05:53:00,961 INFO    : Forward process, Dataset size: 2499, Batch size: 4
'''

### 2.3.3 Train từ epoch 6-9

In [16]:
# Load weights epoch 6
# https://wandb.ai/doanngoccuong_nh/FaceAlignment_ADNetwithSTARloss/artifacts/model/ADNetSTARLoss_bestmodel/epo3to6
# latest
# epo3to6
# v8

import wandb
run = wandb.init()
artifact = run.use_artifact('doanngoccuong_nh/FaceAlignment_ADNetwithSTARloss/ADNetSTARLoss_bestmodel:epo3to6', type='model')
artifact_dir = artifact.download()

wandb: Downloading large artifact ADNetSTARLoss_bestmodel:epo3to6, 236.07MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.8


In [ ]:
%cd /content/drive/MyDrive/colab/FaceAlignment_WFLWds_ADNetwithSTARloss
!python main.py --mode=train --device_ids=0 \
                --image_dir='/content/drive/MyDrive/colab/dataset_for_STAR/image_dir' \
                --annot_dir='/content/drive/MyDrive/colab/dataset_for_STAR/annot_dir' \
                --data_definition=WFLW \
                --batch_size=4 --val_batch_size=4 \
                --pretrained_weight='/content/drive/MyDrive/colab/artifacts/ADNetSTARLoss_bestmodel:v8/best_model_epoch4_valbest_metric0.0744.pkl'

''' INFO - from epo3to5/6 best at 4.
2024-01-06 12:20:39,588 INFO    : Loaded data
2024-01-06 12:20:39,588 INFO    : Optimizer type adam. Start training...
2024-01-06 12:20:39,589 INFO    : Train/Epoch: 0/9, Learning rate decays to [0.001], [Time Left: 00:00:00]eval_time: 0.00,
2024-01-06 12:20:39,589 INFO    : Train/Epoch: 1/9, Learning rate decays to [0.001], [Time Left: 00:00:00]eval_time: 0.00,
2024-01-06 12:20:39,589 INFO    : Train/Epoch: 2/9, Learning rate decays to [0.001], [Time Left: 00:00:00]eval_time: 0.00,
2024-01-06 12:20:39,590 INFO    : Train/Epoch: 3/9, Learning rate decays to [0.001], [Time Left: 00:00:00]eval_time: 0.00,
'''


/content/drive/MyDrive/colab/FaceAlignment_WFLWds_ADNetwithSTARloss
wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /content/drive/MyDrive/colab/FaceAlignment_WFLWds_ADNetwithSTARloss/wandb/run-20240106_122034-pkn67ods
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run elated-microwave-75
wandb: ⭐️ View project at https://wandb.ai/doanngoccuong_nh/FaceAlignment_ADNetwithSTARloss
wandb: 🚀 View run at https://wandb.ai/doanngoccuong_nh/FaceAlignment_ADNetwithSTARloss/runs/pkn67ods
mode is train, config_name is alignment, pretrained_weight is /cont

## 2.4 INFER - PREDECTION - SMALL REAL TESTING

In [15]:
# %cd /content/drive/MyDrive/colab
!git clone https://github.com/DoanNgocCuongBKEGNH/FaceAlignment_WFLWds_ADNetwithSTARloss.git --branch WFLWds_ADNetwithSTARLoss
!pip install -r requirements.txt

Cloning into 'FaceAlignment_WFLWds_ADNetwithSTARloss'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 128 (delta 48), reused 107 (delta 27), pack-reused 0
Receiving objects: 100% (128/128), 1013.05 KiB | 4.24 MiB/s, done.
Resolving deltas: 100% (48/48), done.
ERROR: Operation cancelled by user


In [ ]:
%cd /content
!git clone https://github.com/zhenglinzhou/star

/content
Cloning into 'star'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 72 (delta 8), reused 69 (delta 5), pack-reused 0
Receiving objects: 100% (72/72), 990.84 KiB | 7.86 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
%cd /content
import wandb
wandb.login(key="c8767797aae76cbcd389ff29929ace1ac3021161")
run = wandb.init()
artifact = run.use_artifact('doanngoccuong_nh/FaceAlignment_ADNetwithSTARloss/ADNetSTARLoss_bestmodel:v0', type='model')
artifact_dir = artifact.download()

# https://drive.usercontent.google.com/download?id=1aOx0wYEZUfBndYy_8IYszLPG_D2fhxrT&export=download&authuser=0
# https://drive.usercontent.google.com/download?id=1aOx0wYEZUfBndYy_8IYszLPG_D2fhxrT
# %cd /content/drive/MyDrive/colab
%cd /content
import gdown
gdown.download('https://drive.google.com/u/0/uc?id=1aOx0wYEZUfBndYy_8IYszLPG_D2fhxrT', 'WFLW_STARLoss_NME_4_02_FR_2_32_AUC_0_605.pkl')


In [ ]:
# %cd /content/drive/MyDrive/colab
!git clone https://github.com/italojs/facial-landmarks-recognition

- Thêm đường dẫn landmark vào file demo.py
- Thêm đường dẫn best model vào file demo.py
-

In [ ]:
!pip install gradio

In [ ]:
%cd /content/FaceAlignment_WFLWds_ADNetwithSTARloss
!python demo.py



- output ra 1 đống DEBUG và Figure(640x480) chỉ thấy như này, ko thấy ảnh ra gì cả, dù trong code có plt.show().
Nguyên nhân có thể là do ảnh kích thước to quá.

- Test ko nên chọn ảnh bị che 1 phần mặt


## 2.5 INFER - EVALUATION on BIG TESTSET